In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
from utils.numbers import rarray, ResultsArray,pool
import tensorflow as tf
from numbers import Number
from tensorflow import feature_column
from tensorflow.keras import layers
import tensorflow.keras.backend as bt
from sklearn.model_selection import train_test_split

def sigmoid(x):
    return 1 / (1 + np.e ** -x)


def sigmoid_deriv(x):
    return x * (1 - x)


def binarr2int(arr):
    return int(''.join(np.array(arr).astype(str)), 2)


def binmax(arr):
    return np.array([1 if n > 0 else 0 for n in arr])


class ResultInt(object):
    def __init__(self, value=(1,)):
        self._N = pool(52)
        if not isinstance(value, list) or not isinstance(value, tuple):
            if isinstance(value, Number):
                if value <= 52:
                    value = [value]
                else:
                    value = binarr_to_values(
                        np.array(list(bin(round(value))[2:])).astype(int))
            if isinstance(value, np.ndarray):
                if np.max(value) == 1:
                    value = binarr_to_values(value)
                else:
                    value = value.tolist()
            if isinstance(value, str):
                if len(value) > 3:
                    value = binarr_to_values(
                        np.array(list(value)).astype(int))
        else:
            if len(value) > 12:
                value = binarr_to_values(
                    np.array(list(value)).astype(int))

        self._i = list(map(lambda x: x % 52 if x != 52 else 52, value))

    def __repr__(self):
        return f"{self.__class__.__name__}(N={len(self._N)}, value={self._i})"

    def __str__(self):
        return bin(int(self))[2:]
    @property
    def arr(self):
        return np.where(np.isin(self._N, self._i), 1, 0)

    def __int__(self):
        return int(''.join(np.array(self.arr).astype(str)), 2)

    def __index__(self):
        return int(self)

    def __or__(self, other):
        other = other if isinstance(
            other, self.__class__) else self.__class__(other)
        return self.__class__(int(self) | int(other))

    def __and__(self, other):
        other = other if isinstance(
            other, self.__class__) else self.__class__(other)
        return self.__class__(int(self) & int(other))

    def __not__(self):
        return self.__class__(~int(self))

    def __xor__(self, other):
        other = other if isinstance(
            other, self.__class__) else self.__class__(other)
        return self.__class__(int(self) ^ int(other))
    # def __div__(self, other):
    #     other = other if isinstance(
    #         other, self.__class__) else self.__class__(other)
    #     return np.sum(np.array(list(bin(int(self) & int(other))[2:])).astype(int)) / 6.0

    def __sub__(self, other):
        other = other if isinstance(
            other, self.__class__) else self.__class__(other)
        t, p = (self._i, other._i)
        return self.__class__([dgt for dgt in np.where(np.isin(t, p), 0, p).tolist() if dgt > 0])

    def __add__(self, other):
        other = other if isinstance(
            other, self.__class__) else self.__class__(other)
        return self.__class__(binmax(np.array(self.arr) + np.array(other.arr)))

    def __mul__(self, other):
        other = other if isinstance(
            other, self.__class__) else self.__class__(other)
        return self.__class__(binmax(np.array(self.arr) * np.array(other.arr)))

    # def __idiv__(self, other):
    #     return self.__div__(other)

    def __truediv__(self, other):
        other = other if isinstance(
            other, self.__class__) else self.__class__(other)
        return np.sum(np.array(list(bin(int(self) & int(other))[2:])).astype(int)) / 6.0

    def __itruediv__(self, other):
        return self.__truediv__(other)

    def __isub__(self, other):
        return self.__sub__(other)

    def __iadd__(self, other):
        return self.__add__(other)

    def __imul__(self, other):
        return self.__mul__(other)

def cols2rec(cols=('ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6', 'bonusBall')):
    def _inner(x):
        x['record'] = ResultInt([int(x[c]) for c in cols])
        return x
    return _inner

def cols2nums(cols=('ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6', 'bonusBall')):
    def _inner(x):
        x['record'] = np.array([int(x[c]) for c in cols])
        return x
    return _inner

def as_record(data, cols=('ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6')):
    data['record'] = 0
    fn = cols2rec(cols)
    return data.apply(fn, axis=1)

def train_split(data, split_by=25, cols=('ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6')):
    r, c = data.shape
    chunk_indices = [(k, min(r, k+split_by)-1, min(r, k+split_by))
                     for k in range(0, r-split_by)]    
    inputs = []
    targets = []
    for s, e, t in chunk_indices:
        inputs.append(data[s:e].record.tolist())
        targets.append(data[t:t+1].record)
    ncols = ['R{}'.format(x) for x in range(1, split_by)]
    trgs = pd.concat(targets)
    return pd.DataFrame(inputs, columns=ncols, index=trgs.index), trgs

@tf.function
def accuracy_met(y_true, y_pred):
    #t, p = y_true.numpy(), y_pred.numpy()  
    return tf.sets.size(tf.sets.intersection(y_true, y_pred)) * 1.0/tf.sets.size(y_pred)
  
def create_model(inputs=5, shape=([6, 'relu'], [24, 'relu'], [1,'sigmoid']), 
                loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']):
    model = Sequential()
    i = 0
    #feature_columns = []
    #     for f in ['ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6', ]:
    #         feature_columns.append(feature_column.numeric_column(f, dtype=tf.int32))
        
    #feature_layer = layers.DenseFeatures(feature_columns)
    #model.add(feature_layer)
    for s, a in shape:
        kw = {
            'activation': a
        }
        if i == 0:
           kw['input_dim'] = inputs 
        model.add(Dense(s, **kw))
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

def ttv_data(dataframe, s=0.2):
    train, test = train_test_split(dataframe, test_size=s)
    train, validate = train_test_split(train, test_size=s)
    return train, test, validate

def load_data(fname="data/lotto_daf.pickle", 
              cols=('drawNumber', 'ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6', 'bonusBall')):
    data = [[1994, '2020/02/08',3,41,37,28,51,42,14],[1995, '2020/02/12',16,18,32,38,39,44,31]]
    kw = {
        'columns': 'drawNumber,drawDate,ball1,ball2,ball3,ball4,ball5,ball6,bonusBall'.split(','),
        'index': [n[0] for n in data]
    }
    df = pd.concat([pd.read_pickle(fname),pd.DataFrame(data, **kw)])
    return df
    for c in cols:
        df[c] = df[c].astype(int)
    return df[list(cols)]
    
# def facc():
#     dense = tf.keras.layers.Dense(1, input_dim=6)    
#     loss_fn = lambda: tf.reduce_sum((dense(x) - tf.constant([1., -1.])) ** 2.) 
#     kernel_fprop = [] 
#     with tf.autodiff.ForwardAccumulator( 
#         dense.kernel, tf.constant([0])) as acc: 
#     kernel_fprop.append(acc.jvp(loss_fn())) 

In [ ]:
def df_to_dataset(dataframe, shuffle=False, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('record')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

def get_ttv():
    data_orig = as_record(load_data())
    cols=('drawNumber', 'ball1', 'ball2', 'ball3', 'ball4', 'ball5', 'ball6', 'bonusBall')
    for c in cols:
        data_orig[c] = data_orig[c].astype(int)
    #.loc[df.dra
    main_df = data_orig.copy()
    dfr = main_df.pop('record')
    dfr.index = dfr.index - 1
    main_df = main_df.merge(dfr, how='left', left_index=True, right_index=True)
    dd = main_df.pop('drawDate')
    train, test, val = ttv_data(main_df)
    return train, test, val


In [ ]:
def datint():
    data_orig = load_data()
    data = as_record(data_orig)
    # data['record'] = data['record'].apply(lambda x: np.array(x))
    inpts, trgs = train_split(data, split_by=6)
    x = inpts.to_numpy()
    y = trgs.to_numpy()
    m = int(len(x)/ 2)
    x, test_x = (x[:m], x[m:])
    y, test_y = (y[:m], y[m:])
    return [x, y, test_x, test_y]
    
x, y, test_x, test_y = datint()

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model = create_model(inputs=5,optimizer=optimizer,metrics=['binary_accuracy'])

In [200]:
print(model.summary())


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 6)                 36        
_________________________________________________________________
dense_82 (Dense)             (None, 24)                168       
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 25        
Total params: 229
Trainable params: 229
Non-trainable params: 0
_________________________________________________________________
None


In [201]:
# import tempfile
# model_dir = tempfile.mkdtemp()
# keras_estimator = tf.keras.estimator.model_to_estimator(
#     keras_model=model, model_dir=model_dir)
xt,yt = x.astype(int), y.astype(int)
xt,yt 

(array([[    274883837952,     141745455112,  141355967840276,
            4449598702080,      10873745408],
        [    141745455112,  141355967840276,    4449598702080,
              10873745408,  167400779546632],
        [ 141355967840276,    4449598702080,      10873745408,
          167400779546632,    4707288350740],
        ...,
        [   8869107466818,      77319897156, 2256203229167618,
            8813272925824, 1125899932271680],
        [     77319897156, 2256203229167618,    8813272925824,
         1125899932271680,     158997676033],
        [2256203229167618,    8813272925824, 1125899932271680,
             158997676033,  567350418079744]]),
 array([   4707288350740,    2270024564768,     274880037385,
             68719476978,  141012940881936,       1627398150,
           4569845209216,       3223453776,  356791527931904,
           2216211726336,    4681648570404,     691500220544,
            146297405568,  140806478432256,      19613089824,
           4402349900

In [202]:
model.fit(xt[:12],yt[:12],epochs=250)

Train on 12 samples
Epoch 1/250
12/12 [==============================] - 0s 15ms/sample - loss: -644253274644069038570864640.0000 - binary_accuracy: 0.0000e+00
Epoch 2/250
12/12 [==============================] - 0s 111us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 3/250
12/12 [==============================] - 0s 123us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 4/250
12/12 [==============================] - 0s 94us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 5/250
12/12 [==============================] - 0s 97us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 6/250
12/12 [==============================] - 0s 258us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 7/250
12/12 [==============================] - 0s 167us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 8/250
12/12 [==============================] - 0s 121us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 9/250
12/12 [==============================] - 0s 123us/sam

Epoch 74/250
12/12 [==============================] - 0s 100us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 75/250
12/12 [==============================] - 0s 96us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 76/250
12/12 [==============================] - 0s 96us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 77/250
12/12 [==============================] - 0s 100us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 78/250
12/12 [==============================] - 0s 99us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 79/250
12/12 [==============================] - 0s 98us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 80/250
12/12 [==============================] - 0s 98us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 81/250
12/12 [==============================] - 0s 107us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 82/250
12/12 [==============================] - 0s 113us/sample - loss: nan - binary_accuracy: 0.0000e+

Epoch 147/250
12/12 [==============================] - 0s 107us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 148/250
12/12 [==============================] - 0s 91us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 149/250
12/12 [==============================] - 0s 91us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 150/250
12/12 [==============================] - 0s 88us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 151/250
12/12 [==============================] - 0s 103us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 152/250
12/12 [==============================] - 0s 100us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 153/250
12/12 [==============================] - 0s 116us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 154/250
12/12 [==============================] - 0s 126us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 155/250
12/12 [==============================] - 0s 154us/sample - loss: nan - binary_accurac

Epoch 220/250
12/12 [==============================] - 0s 108us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 221/250
12/12 [==============================] - 0s 89us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 222/250
12/12 [==============================] - 0s 104us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 223/250
12/12 [==============================] - 0s 110us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 224/250
12/12 [==============================] - 0s 114us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 225/250
12/12 [==============================] - 0s 149us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 226/250
12/12 [==============================] - 0s 143us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 227/250
12/12 [==============================] - 0s 124us/sample - loss: nan - binary_accuracy: 0.0000e+00
Epoch 228/250
12/12 [==============================] - 0s 109us/sample - loss: nan - binary_accur

In [46]:
l = test_x[-1:]
#val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
predictions = model.predict(l)

yo, accuracy = model.evaluate(test_x, test_y)
print('Accuracy: %.2f' % (accuracy*1000000000000000))

ValueError: Error when checking input: expected dense_51_input to have shape (24,) but got array with shape (1,)

In [136]:
test_ds.take(1)

<TakeDataset shapes: ({drawNumber: (None,), ball1: (None,), ball2: (None,), ball3: (None,), ball4: (None,), ball5: (None,), ball6: (None,), bonusBall: (None,)}, (None, 6)), types: ({drawNumber: tf.int32, ball1: tf.int32, ball2: tf.int32, ball3: tf.int32, ball4: tf.int32, ball5: tf.int32, ball6: tf.int32, bonusBall: tf.int32}, tf.int32)>

In [129]:
predictions

array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]], dtype=float32)